# Work in Progress/Add in Future:

- Scrape data for min. 1 week
- Change cities

In [1]:
import os
import sys
from datetime import datetime as dt
from datetime import date
from datetime import timedelta
import glob
import numpy as np
import pandas as pd
sys.path.insert(0, os.path.abspath('..\modules'))
import pricemate as pm
todaydate = date.today().strftime('%Y%m%d')

In [2]:
# scrape data using pricemate module
df_today = pm.multiple_days_df(0,20)


Looking for [chromedriver 79.0.3945.36 win32] driver in cache 
File found in cache by path [C:\Users\Zenbook\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.exe]


In [3]:
df_today.head()

,arrival_date,arrival_time,depart_date,depart_time,price,seats,tclass,title
0,2020-01-13,01:27,2020-01-12,22:05,110000,99,ekonomi (Subclass S),argo parahyangan 50
1,2020-01-13,02:05,2020-01-12,22:55,110000,99,ekonomi (Subclass S),argo parahyangan 52
2,2020-01-13,02:40,2020-01-12,23:30,110000,99,ekonomi (Subclass S),argo parahyangan 70F
3,2020-01-13,01:27,2020-01-12,22:05,150000,99,eksekutif (Subclass J),argo parahyangan 50
4,2020-01-13,02:05,2020-01-12,22:55,150000,99,eksekutif (Subclass J),argo parahyangan 52


In [4]:
df_today['scrape_date'] = dt.date(dt.now())

# export scrape output to new csv
df_today.to_csv('..\scrape-csv\gmr-bdg-' + todaydate + '.csv', index=False)

In [5]:
df = df_today
df_today.head()

,arrival_date,arrival_time,depart_date,depart_time,price,seats,tclass,title,scrape_date
0,2020-01-13,01:27,2020-01-12,22:05,110000,99,ekonomi (Subclass S),argo parahyangan 50,2020-01-12
1,2020-01-13,02:05,2020-01-12,22:55,110000,99,ekonomi (Subclass S),argo parahyangan 52,2020-01-12
2,2020-01-13,02:40,2020-01-12,23:30,110000,99,ekonomi (Subclass S),argo parahyangan 70F,2020-01-12
3,2020-01-13,01:27,2020-01-12,22:05,150000,99,eksekutif (Subclass J),argo parahyangan 50,2020-01-12
4,2020-01-13,02:05,2020-01-12,22:55,150000,99,eksekutif (Subclass J),argo parahyangan 52,2020-01-12


In [6]:
# import and combine past scrape data with today's scrape, if any

files = glob.glob('..\scrape-csv\*.csv')
for file in files:
    scrapedate = file[-12:-4]
    if scrapedate < todaydate:
        past_df = pd.read_csv(file)
        past_df['scrape_date'] = pd.to_datetime(scrapedate)
        df = df.append(past_df)


# print('Earliest scrape: ' + str(min(df['scrape_datetime'])))
# print('Latest scrape: ' + str(max(df['scrape_datetime'])))

C:\Users\Zenbook\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [7]:
# change or remove columns
"""
for file in files:
    past_df = pd.read_csv(file)
    if 'arrive_time' in past_df.columns:
        past_df['arrival_time'] = past_df['arrive_time']
        past_df = past_df.drop(columns = ['arrive_time'])
        past_df.to_csv(file, index=False)
"""

"\nfor file in files:\n    past_df = pd.read_csv(file)\n    if 'arrive_time' in past_df.columns:\n        past_df['arrival_time'] = past_df['arrive_time']\n        past_df = past_df.drop(columns = ['arrive_time'])\n        past_df.to_csv(file, index=False)\n"

In [8]:
# duplicate combined dataframe
df_raw = df
df.head()

,arrival_date,arrival_datetime,arrival_time,depart_date,depart_datetime,depart_time,price,scrape_date,seats,tclass,title
0,2020-01-13,NaN,01:27,2020-01-12,NaN,22:05,110000,2020-01-12,99,ekonomi (Subclass S),argo parahyangan 50
1,2020-01-13,NaN,02:05,2020-01-12,NaN,22:55,110000,2020-01-12,99,ekonomi (Subclass S),argo parahyangan 52
2,2020-01-13,NaN,02:40,2020-01-12,NaN,23:30,110000,2020-01-12,99,ekonomi (Subclass S),argo parahyangan 70F
3,2020-01-13,NaN,01:27,2020-01-12,NaN,22:05,150000,2020-01-12,99,eksekutif (Subclass J),argo parahyangan 50
4,2020-01-13,NaN,02:05,2020-01-12,NaN,22:55,150000,2020-01-12,99,eksekutif (Subclass J),argo parahyangan 52


## Data Preprocessing
- Add preprocessing here based on diagnostic testing

In [9]:
# concat date and time, then convert to datetime
df['arrival_datetime'] = pd.to_datetime(df['arrival_date'] + ' ' + df['arrival_time'])
df['depart_datetime'] = pd.to_datetime(df['depart_date'] + ' ' + df['depart_time'])

In [10]:
def remove_nonnumeric(string):
    if isinstance(string, str):
        string = ''.join(filter(lambda x: x.isdigit(), string))
    return string

In [11]:
df['price'] = df['price'].apply(remove_nonnumeric).apply(pd.to_numeric)
df['seats'] = df['seats'].apply(remove_nonnumeric).apply(pd.to_numeric)

In [12]:
three_dates = ['arrival_date', 'depart_date', 'scrape_date']
df[three_dates] = df[three_dates].apply(pd.to_datetime)

In [13]:
# does arrival and depart time need to be converted into dt.time?
# df['arrival_time'] = pd.to_datetime(df['arrival_time']).dt.time
# df['depart_time'] = pd.to_datetime(df['depart_time']).dt.time

## Diagnostic assumptions/test suite
- Arrival date >= depart date
- depart date >= scrape date
- 0 <= seats <= 99


In [14]:
assert min(df['arrival_datetime'] - df['depart_datetime']) > timedelta(0), 'arrival before departure exists'
assert min(df['depart_date'] - df['scrape_date']) >= timedelta(0), 'departure before scrape exists'
assert min(df['seats']) >= 0, 'negative seats exist'
assert max(df['seats']) <= 99, '>99 seats exist'

## View basics of data

In [15]:
df.head()

,arrival_date,arrival_datetime,arrival_time,depart_date,depart_datetime,depart_time,price,scrape_date,seats,tclass,title
0,2020-01-13,2020-01-13 01:27:00,01:27,2020-01-12,2020-01-12 22:05:00,22:05,110000.0,2020-01-12,99,ekonomi (Subclass S),argo parahyangan 50
1,2020-01-13,2020-01-13 02:05:00,02:05,2020-01-12,2020-01-12 22:55:00,22:55,110000.0,2020-01-12,99,ekonomi (Subclass S),argo parahyangan 52
2,2020-01-13,2020-01-13 02:40:00,02:40,2020-01-12,2020-01-12 23:30:00,23:30,110000.0,2020-01-12,99,ekonomi (Subclass S),argo parahyangan 70F
3,2020-01-13,2020-01-13 01:27:00,01:27,2020-01-12,2020-01-12 22:05:00,22:05,150000.0,2020-01-12,99,eksekutif (Subclass J),argo parahyangan 50
4,2020-01-13,2020-01-13 02:05:00,02:05,2020-01-12,2020-01-12 22:55:00,22:55,150000.0,2020-01-12,99,eksekutif (Subclass J),argo parahyangan 52


In [16]:
df.describe()

,price,seats
count,3584.000000,9508.000000
mean,167945.033482,35.573412
std,66595.659819,47.397932
min,110000.000000,0.000000
25%,110000.000000,0.000000
50%,150000.000000,0.000000
75%,170000.000000,99.000000
max,375000.000000,99.000000


In [17]:
df.dtypes
# all data types seem to be objects

arrival_date        datetime64[ns]
arrival_datetime    datetime64[ns]
arrival_time                object
depart_date         datetime64[ns]
depart_datetime     datetime64[ns]
depart_time                 object
price                      float64
scrape_date         datetime64[ns]
seats                        int64
tclass                      object
title                       object
dtype: object

## Exploratory questions & assumptions
1. which times, days, classes are the least available?
2. When do the least available tickets start becoming unavailable?

Target variable: % unavailable tickets on a day/time/class 
= (Seats == 0) / (# seats in group)

### Start with today's scrape data only

In [51]:
# average availability over next 20 days
df_today['has_seats'] = df_today['seats'].apply(pd.to_numeric) > 0
df_today['has_seats'].mean()

0.38095238095238093

Only 38% of trains had available tickets

In [53]:
# grouped per day
df_today.groupby('depart_date')['has_seats'].mean()

depart_date
2020-01-12    0.269231
2020-01-13    0.350877
2020-01-14    0.401869
2020-01-15    0.401869
2020-01-16    0.412844
2020-01-17    0.216216
2020-01-18    0.379310
2020-01-19    0.413793
2020-01-20    0.403509
2020-01-21    0.401869
2020-01-22    0.401869
2020-01-23    0.412844
2020-01-24    0.279279
2020-01-25    0.396552
2020-01-26    0.413793
2020-01-27    0.403509
2020-01-28    0.401869
2020-01-29    0.401869
2020-01-30    0.412844
2020-01-31    0.342342
2020-02-01    0.398230
Name: has_seats, dtype: float64

after next two days, average avaialiblity = 40%